
# --- Boltz2: Protein Structure Prediction Pipeline ---

![Python](https://img.shields.io/badge/Python-3.10-blue?logo=python)
![CUDA](https://img.shields.io/badge/CUDA-Enabled-green?logo=nvidia)
![Boltz2](https://img.shields.io/badge/Model-Boltz2-purple)
![Platform](https://img.shields.io/badge/Platform-Colab%20|%20Linux-lightgrey?logo=googlecolab)
![License](https://img.shields.io/badge/License-MIT-orange)
![Status](https://img.shields.io/badge/Status-Active-success)
![Build](https://img.shields.io/badge/Build-Stable-brightgreen)
![Contributions](https://img.shields.io/badge/Contributions-Welcome-blue)
<br>

---

## Boltz2: Deep Learning Pipeline for Protein Structure Prediction

Boltz2 is an **open-source, deep learning-based software** for predicting **3D protein structures** from amino acid sequences.  
It leverages **advanced neural networks** and **diffusion models** to generate accurate protein models, supporting both **monomeric** and **complex assemblies**.

---

###  Pipeline Overview
1. **Input**: Provide a protein sequence (and optional ligands).  
2. **YAML Generation**: The sequence is formatted into a YAML config.  
3. **MSA Search**: Boltz2 fetches multiple sequence alignments (MSA) using online servers.  
4. **Structure Prediction**: The neural network predicts 3D coordinates using diffusion and recycling steps.  
5. **Output**: Results include 3D models (CIF/PDB), confidence scores (**pLDDT**), and error heatmaps (**PAE**).  
6. **Visualization**: The notebook displays the predicted structure and confidence plots.  

---

 **Note:** This notebook automates the full Boltz2 workflow, from setup to visualization, with **color-coded status** and **interactive outputs**.  

---

##  Credits & Authorship

- **Notebook Developer:** Atharva Tilewale  
- **Affiliation:** Gujarat Biotechnology University | Bioinformatics & Computational Biology  
- **GitHub Repository:** [Boltz-Notebook](https://github.com/AtharvaTilewale/Boltz-Notebook)  
- **Contact:** [LinkedIn](https://www.linkedin.com/in/atharvatilewale) | [GitHub](https://github.com/AtharvaTilewale)  

**Acknowledgements:**  
- **Boltz2 framework**: [Original Boltz repository](https://github.com/jwohlwend/boltz) by J. Wohlwend and collaborators.  
- **Dependencies:** PyTorch, Biopython, NumPy, Matplotlib, Py3Dmol, PyYAML.  
- Special thanks to the **open-source community** for providing tools that make structural bioinformatics more accessible.  

---

## References

- Passaro, S., Corso, G., Wohlwend, J., Reveiz, M., Thaler, S., Somnath, V. R., Getz, N., Portnoi, T., Roy, J., Stark, H., Kwabi-Addo, D., Beaini, D., Jaakkola, T., & Barzilay, R. (2025).  
  **Boltz-2: Towards Accurate and Efficient Binding Affinity Prediction.** *bioRxiv.*  
    [![bioRxiv Boltz2](https://img.shields.io/badge/bioRxiv-Boltz2-red)](https://doi.org/10.1101/2025.06.14.659707) 

- Wohlwend, J., Corso, G., Passaro, S., Getz, N., Reveiz, M., Leidal, K., Swiderski, W., Atkinson, L., Portnoi, T., Chinn, I., Silterra, J., Jaakkola, T., & Barzilay, R. (2024).  
  **Boltz-1: Democratizing Biomolecular Interaction Modeling.** *bioRxiv.*  
    [![bioRxiv Boltz1](https://img.shields.io/badge/bioRxiv-Boltz1-orange)](https://doi.org/10.1101/2024.11.19.624167)

- Mirdita, M., Schütze, K., Moriwaki, Y., Heo, L., Ovchinnikov, S., & Steinegger, M. (2022).  
  **ColabFold: Making protein folding accessible to all.** *Nature Methods.*  
    [![ColabFold](https://img.shields.io/badge/ColabFold-Reference-yellow)](https://doi.org/10.1038/s41592-022-01488-1)

---

## Cite
If you use this notebook, please **cite the following repository**:

[![GitHub Repo](https://img.shields.io/badge/GitHub-Boltz--Notebook-181717?logo=github)](https://github.com/AtharvaTilewale/Boltz-Notebook)

In [ ]:
# @title Install Dependencies and Boltz2 with CUDA support 
import sys
import subprocess
import threading
import time
import os
import shutil

# ANSI color codes for colored output 
class Color:
    CYAN = "\033[96m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    RED = "\033[91m"
    RESET = "\033[0m"

repo_dir = "boltz"

steps = [
    {
        "loader": f"{Color.CYAN}Cloning repository...{Color.RESET}",
        "done": f"[{Color.GREEN}✔{Color.RESET}] Repository cloned successfully.",
        "fail": f"[{Color.RED}✘{Color.RESET}] Repository clone failed.",
        "cmd": ["git", "clone", "https://github.com/jwohlwend/boltz.git"]
    },
    {
        "loader": f"{Color.RESET}Installing dependencies...{Color.RESET}",
        "done": f"[{Color.GREEN}✔{Color.RESET}] Dependencies installed successfully.",
        "fail": f"[{Color.RED}✘{Color.RESET}] Dependency installation failed.",
        "cmd": [sys.executable, "-m", "pip", "install", "-e", "boltz[cuda]", "biopython", "numpy", "matplotlib", "pyyaml", "py3Dmol", "--quiet"]
    },
    {
        "loader": f"{Color.CYAN}Validating installation...{Color.RESET}",
        "done": f"[{Color.GREEN}✔{Color.RESET}] Validation complete.",
        "fail": f"[{Color.RED}✘{Color.RESET}] Validation failed.",
        "cmd": [sys.executable, "-c", "import torch; print('Torch CUDA available:', torch.cuda.is_available()); print('CUDA device count:', torch.cuda.device_count())"]
    }
]

def loader(msg, stop_event):
    symbols = ["-", "\\", "|", "/"]
    i = 0
    while not stop_event.is_set():
        sys.stdout.write(f"\r[{symbols[i % len(symbols)]}] {msg}   ")
        sys.stdout.flush()
        time.sleep(0.1)
        i += 1
    sys.stdout.write("\r" + " " * (len(msg) + 10) + "\r")

# Step 1: Remove repo if it exists
if os.path.isdir(repo_dir):
    print(f"{Color.YELLOW}[i] Repository already exists. Removing '{repo_dir}'...{Color.RESET}")
    try:
        shutil.rmtree(repo_dir)
        print(f"[{Color.GREEN}✔{Color.RESET}] Existing repository removed.")
    except Exception as e:
        print(f"[{Color.RED}✘{Color.RESET}] Failed to remove existing repository: {e}")
        raise

all_success = True

# Main steps
for step in steps:
    stop_event = threading.Event()
    t = threading.Thread(target=loader, args=(step["loader"], stop_event))
    t.start()
    try:
        subprocess.run(step["cmd"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
        stop_event.set()
        t.join()
        print(step["done"])
    except Exception as e:
        stop_event.set()
        t.join()
        print(f"{step['fail']} {e}")
        all_success = False
        break

if all_success:
    print(f"{Color.GREEN}All steps completed successfully.{Color.RESET}")


In [ ]:

# @title Download CCD Dataset and Test Boltz2
import sys
import threading
import time
import os

# ANSI color codes for colored output
class Color:
    CYAN = "\033[96m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    RED = "\033[91m"
    RESET = "\033[0m"

def loader(msg, stop_event):
    symbols = ["-", "\\", "|", "/"]
    i = 0
    while not stop_event.is_set():
        sys.stdout.write(f"\r[{symbols[i % len(symbols)]}] {msg}   ")
        sys.stdout.flush()
        time.sleep(0.1)
        i += 1
    sys.stdout.write("\r" + " " * (len(msg) + 10) + "\r")
    sys.stdout.flush()

# Step 1: Create data directory
os.makedirs("/content/boltz_data", exist_ok=True)

# Step 2: Write YAML file
yaml_content = f"""\
version: 1
sequences:
- protein:
  id: [A, B]
  sequence: MVTPEGNVSLVDESLLVGVTDEDRAVRSAHQF
- ligand:
  id: [C, D]
  ccd: SAH   # fetch ligand from CCD
- ligand:
  id: [E, F]
  smiles: 'N[C@@H](Cc1ccc(O)cc1)C(=O)O'
"""
with open("/content/boltz_data/test.yaml", "w") as f:
    f.write(yaml_content)

# Step 3: Run boltz predict (silent)
step_msg = f"{Color.YELLOW}Downloading CCD Dataset...{Color.RESET}"
stop_event = threading.Event()
t = threading.Thread(target=loader, args=(step_msg, stop_event))
t.start()
try:
    import subprocess
    subprocess.run(
        ["boltz", "predict", "test.yaml", "--use_msa_server"],
        cwd="/content/boltz_data",
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
        check=True
    )
    stop_event.set()
    t.join()
    print(f"[{Color.GREEN}✔{Color.RESET}] CCD Dataset Downloaded and validated.")
except Exception as e:
    stop_event.set()
    t.join()
    print(f"[{Color.RED}✘{Color.RESET}] CCD Dataset Download or validation failed: {e}")


In [ ]:
# Colab HTML UI -> Python YAML saver (first protein block permanent)

from IPython.display import HTML, display
import yaml
from google.colab import output

# Python callback to save YAML
def _save_params(data):
    if not isinstance(data, dict) or 'sequences' not in data:
        return {'status': 'error', 'message': 'Invalid data structure'}

    filename = "params.yaml"
    try:
        with open(filename, 'w') as f:
            yaml.dump({
                'version': 1,   # always set version=1
                'sequences': data['sequences']
            }, f, sort_keys=False)
        return {'status': 'ok', 'filename': filename}
    except Exception as e:
        return {'status': 'error', 'message': str(e)}

output.register_callback('save_params', _save_params)

# HTML + JS
html = r"""
<style>
  body { background:#fff; color:#333; font-family:Arial, sans-serif; padding: 20px }
  .block { border: 1px solid #bbb; padding: 14px; margin:10px 0; border-radius:8px; background:#fff; box-shadow:0 1px 3px rgba(0,0,0,0.05); }
  .row { display:flex; gap:10px; align-items:center; margin-bottom:8px; }
  .row input[type="text"], .row textarea, .row select { 
    flex:1; padding:8px; border:1px solid #ccc; border-radius:6px; 
    font-size:14px; color:#333; background:#fafafa; 
  }
  .controls { margin-top:14px; display:flex; gap:8px; flex-wrap:wrap; }
  .btn { background:#1976d2; color:#fff; border:none; padding:8px 14px; border-radius:6px; cursor:pointer; font-size:14px; }
  .btn.secondary { background:#555; }
  .small { padding:6px 10px; font-size:13px; }
  .remove-btn { background:#d32f2f; color:#fff; border:none; padding:6px 10px; border-radius:6px; cursor:pointer; margin-left:12px; }
  .title { font-weight:600; margin-bottom:6px; font-size:15px; }
</style>

<div>
  <div id="sequences_container"></div>

  <div class="controls">
    <button class="btn small" onclick="addProtein()">+ Add Protein</button>
    <button class="btn small" onclick="addLigand()">+ Add Ligand</button>
    <button class="btn secondary small" onclick="clearAll()">Clear All</button>
    <button class="btn small" id="saveBtn" onclick="save()">Save YAML</button>
    <span id="status" style="margin-left:12px"></span>
  </div>

  <!-- Template for the FIRST protein (no remove button) -->
  <template id="first_protein_template">
    <div class="block seq-block first-protein" data-type="protein">
      <div style="display:flex; justify-content:space-between; align-items:center; margin-bottom:8px;">
        <div class="title">Protein (Primary)</div>
      </div>
      <div class="row"><label style="width:120px">IDs (comma):</label><input class="p-ids" type="text" placeholder="A,B" /></div>
      <div><label style="display:block; margin-bottom:4px">Sequence:</label>
        <textarea class="p-seq" rows="6" style="width:100%;"></textarea>
      </div>
    </div>
  </template>

  <!-- Template for extra proteins -->
  <template id="protein_template">
    <div class="block seq-block" data-type="protein">
      <div style="display:flex; justify-content:space-between; align-items:center; margin-bottom:8px;">
        <div class="title">Protein</div>
        <button class="remove-btn" onclick="removeBlock(this)">Remove</button>
      </div>
      <div class="row"><label style="width:120px">IDs (comma):</label><input class="p-ids" type="text" placeholder="C,D" /></div>
      <div><label style="display:block; margin-bottom:4px">Sequence:</label>
        <textarea class="p-seq" rows="6" style="width:100%;"></textarea>
      </div>
    </div>
  </template>

  <!-- Template for ligands -->
  <template id="ligand_template">
    <div class="block seq-block" data-type="ligand">
      <div style="display:flex; justify-content:space-between; align-items:center; margin-bottom:8px;">
        <div class="title">Ligand</div>
        <button class="remove-btn" onclick="removeBlock(this)">Remove</button>
      </div>
      <div class="row"><label style="width:120px">IDs (comma):</label><input class="l-ids" type="text" placeholder="E,F" /></div>
      <div class="row"><label style="width:120px">Type:</label>
        <select class="l-type" onchange="onLigandTypeChange(this)">
          <option value="ccd">CCD</option>
          <option value="smiles">SMILES</option>
        </select>
      </div>
      <div class="row lig-value-row">
        <label style="width:120px">Value:</label><input class="l-value" type="text" placeholder="SAH or SMILES" />
      </div>
    </div>
  </template>
</div>

<script>
  function addProtein(first=false) {
    const tpl = document.getElementById(first ? 'first_protein_template' : 'protein_template');
    const node = tpl.content.cloneNode(true);
    document.getElementById('sequences_container').appendChild(node);
  }

  function addLigand() {
    const tpl = document.getElementById('ligand_template');
    const node = tpl.content.cloneNode(true);
    document.getElementById('sequences_container').appendChild(node);
  }

  function removeBlock(btn) {
    const block = btn.closest('.seq-block');
    if (block && !block.classList.contains('first-protein')) block.remove();
  }

    function clearAll() {
    const container = document.getElementById('sequences_container');
    const first = container.querySelector('.first-protein');
    container.innerHTML = '';

    if (first) {
      // reset all inputs inside first protein block
      first.querySelectorAll('input, textarea').forEach(el => el.value = '');
      container.appendChild(first);  // keep first protein
    }

    document.getElementById('status').innerText = '';
  }


  function onLigandTypeChange(select) {
    const block = select.closest('.seq-block');
    const valueInput = block.querySelector('.l-value');
    if (select.value === 'ccd') valueInput.placeholder = 'e.g., SAH';
    else valueInput.placeholder = 'e.g., CCO... (SMILES)';
  }

  async function save() {
    const statusEl = document.getElementById('status');
    statusEl.innerText = 'Saving...';

    const sequences = [];
    const blocks = document.querySelectorAll('.seq-block');
    blocks.forEach(b => {
      const type = b.dataset.type;
      if (type === 'protein') {
        const ids = (b.querySelector('.p-ids').value || '').split(',').map(s=>s.trim()).filter(Boolean);
        const seq = b.querySelector('.p-seq').value || '';
        sequences.push({ protein: { id: ids, sequence: seq } });
      } else if (type === 'ligand') {
        const ids = (b.querySelector('.l-ids').value || '').split(',').map(s=>s.trim()).filter(Boolean);
        const ltype = b.querySelector('.l-type').value;
        const lvalue = b.querySelector('.l-value').value || '';
        const entry = { id: ids };
        if (ltype === 'ccd') entry.ccd = lvalue;
        else entry.smiles = lvalue;
        sequences.push({ ligand: entry });
      }
    });

    const payload = { sequences: sequences };

    try {
      const result = await google.colab.kernel.invokeFunction('save_params', [payload], {});
      if (result && result.status === 'ok') {
        statusEl.innerHTML = '✅ Saved as <b>' + result.filename + '</b>. You can download it from the Files panel.';
      } else {
        statusEl.innerText = 'Error: ' + (result && result.message ? result.message : 'unknown');
      }
    } catch (err) {
      statusEl.innerText = 'Error saving: ' + err;
    }
  }

  // show the permanent first protein block initially
  addProtein(true);
</script>
<script>
  async function save() {
    const statusEl = document.getElementById('status');
    statusEl.innerText = 'Validating...';

    const sequences = [];
    const blocks = document.querySelectorAll('.seq-block');
    let valid = true;

    blocks.forEach((b, idx) => {
      const type = b.dataset.type;
      if (type === 'protein') {
        const ids = (b.querySelector('.p-ids').value || '').split(',')
                     .map(s=>s.trim()).filter(Boolean);
        const seq = b.querySelector('.p-seq').value.trim();

        if (ids.length === 0 || !seq) {
          valid = false;
          statusEl.innerText = `⚠️ Protein block ${idx+1}: Please fill both IDs and Sequence.`;
          return;
        }
        sequences.push({ protein: { id: ids, sequence: seq } });

      } else if (type === 'ligand') {
        const ids = (b.querySelector('.l-ids').value || '').split(',')
                     .map(s=>s.trim()).filter(Boolean);
        const ltype = b.querySelector('.l-type').value;
        const lvalue = b.querySelector('.l-value').value.trim();

        if (ids.length === 0 || !lvalue) {
          valid = false;
          statusEl.innerText = `⚠️ Ligand block ${idx+1}: Please fill both IDs and Value.`;
          return;
        }
        const entry = { id: ids };
        if (ltype === 'ccd') entry.ccd = lvalue;
        else entry.smiles = lvalue;
        sequences.push({ ligand: entry });
      }
    });

    if (!valid) return;

    const payload = { sequences: sequences };

    try {
      const result = await google.colab.kernel.invokeFunction('save_params', [payload], {});
      if (result && result.status === 'ok') {
        statusEl.innerHTML = '✅ Saved as <b>' + result.filename + '</b>. You can download it from the Files panel.';
      } else {
        statusEl.innerText = 'Error: ' + (result && result.message ? result.message : 'unknown');
      }
    } catch (err) {
      statusEl.innerText = 'Error saving: ' + err;
    }
  }
</script>


"""

display(HTML(html))


In [ ]:

# @title Boltz2 Engine
import sys
import threading
import time
import os
import re
import shutil
import numpy as np
import matplotlib.pyplot as plt
from Bio.PDB import MMCIFParser, PDBIO
import py3Dmol
import subprocess

# ANSI color codes for colored output
class Color:
    CYAN = "\033[96m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    RED = "\033[91m"
    BLUE = "\033[94m"
    MAGENTA = "\033[95m"
    RESET = "\033[0m"

def loader(msg, stop_event):
    symbols = ["⠋","⠙","⠹","⠸","⠼","⠴","⠦","⠧","⠇","⠏"]
    i = 0
    while not stop_event.is_set():
        sys.stdout.write(f"\r[{symbols[i % len(symbols)]}] {msg}   ")
        sys.stdout.flush()
        time.sleep(0.1)
        i += 1
    sys.stdout.write("\r" + " " * (len(msg) + 10) + "\r")
    sys.stdout.flush()

# Set up parameters
os.chdir("/content/boltz_data/")
protein_seq = "MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQVGQVELGGGPGAGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN"  # @param {type:"string"}
output_dir = "Insulin" # @param {type:"string"}
recycling_steps = 3   # @param {type:"integer"}
diffusion_samples = 1  # @param {type:"integer"}
use_potentials = True  # @param {type:"boolean"}
override = True  # @param {type:"boolean"}

output_path = f"/content/boltz_data/{output_dir}"
if os.path.exists(output_path):
    shutil.rmtree(output_path)

protein_seq = protein_seq.upper()
protein_seq = re.sub(r"\s+", "", protein_seq.strip())

# Write YAML file
yaml_content = f"""\
version: 1
sequences:
  - protein:
      id: A
      sequence: {protein_seq}
"""
with open("protein.yaml", "w") as f:
    f.write(yaml_content)
print(f"[{Color.GREEN}✔{Color.RESET}] Parameters generated")

# Prepare command
cmd = [
    "boltz", "predict", "protein.yaml",
    "--use_msa_server",
    "--out_dir", output_dir,
    "--recycling_steps", str(recycling_steps),
    "--diffusion_samples", str(diffusion_samples),
]
if use_potentials: cmd.append("--use_potentials")
if override: cmd.append("--override")

# Run boltz predict with loader and silent output
step_msg = f"{Color.RESET}Running Boltz2 prediction...{Color.RESET}"
stop_event = threading.Event()
t = threading.Thread(target=loader, args=(step_msg, stop_event))
t.start()
try:
    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
    stop_event.set()
    t.join()
    print(f"[{Color.GREEN}✔{Color.RESET}] Boltz2 run finished successfully!")
except Exception as e:
    stop_event.set()
    t.join()
    print(f"[{Color.RED}✘{Color.RESET}] Boltz2 run failed: {e}")

# Visualize results (with colored section titles and dividers)
def visualize_boltz_results(output_dir, model_id=0):
    """
    Visualize Boltz2 results:
      - 3D protein structure
      - pLDDT confidence plot
      - PAE heatmap
    """
    cif_file = f"/content/boltz_data/{output_dir}/boltz_results_protein/predictions/protein/protein_model_{model_id}.cif"
    pdb_file = f"/content/boltz_data/{output_dir}_model{model_id}.pdb"
    plddt_file = f"/content/boltz_data/{output_dir}/boltz_results_protein/predictions/protein/plddt_protein_model_{model_id}.npz"
    pae_file   = f"/content/boltz_data/{output_dir}/boltz_results_protein/predictions/protein/pae_protein_model_{model_id}.npz"

    # --- Convert CIF to PDB ---
    parser = MMCIFParser(QUIET=True)
    structure = parser.get_structure("protein", cif_file)
    io = PDBIO()
    io.set_structure(structure)
    io.save(pdb_file)

    # --- Load PDB into py3Dmol viewer ---
    with open(pdb_file, "r") as f:
        pdb_data = f.read()

    # --- 3D Structure Viewer ---
    print(f"\n{Color.CYAN}{'='*50}{Color.RESET}")
    print(f"{Color.MAGENTA}3D Protein Structure{Color.RESET}")
    print(f"{Color.RESET}Shows the predicted protein fold as a cartoon model.\nColors follow spectrum by residue index.{Color.RESET}")
    print(f"{Color.CYAN}{'='*50}{Color.RESET}\n")

    viewer = py3Dmol.view(width=600, height=500)
    viewer.addModel(pdb_data, "pdb")
    viewer.setStyle({"cartoon": {"color": "spectrum"}})
    viewer.zoomTo()
    viewer.show()

    # --- pLDDT Plot ---
    print(f"\n{Color.CYAN}{'='*50}{Color.RESET}")
    print(f"{Color.MAGENTA}Predicted Local Distance Difference Test (pLDDT){Color.RESET}")
    print(f"{Color.RESET}Confidence score per residue: Higher = more reliable structure.{Color.RESET}")
    print(f"{Color.CYAN}{'='*50}{Color.RESET}\n")

    plddt = np.load(plddt_file)["plddt"]
    plt.figure(figsize=(10,4))
    plt.plot(plddt, label="pLDDT", color="blue")
    plt.xlabel("Residue index")
    plt.ylabel("pLDDT score")
    plt.title(f"Model {model_id} | Confidence per residue")
    plt.legend()
    plt.tight_layout(pad=3.0)
    plt.show()

    # --- PAE Heatmap ---
    print(f"\n{Color.CYAN}{'='*50}{Color.RESET}")
    print(f"{Color.MAGENTA}Predicted Aligned Error (PAE) Heatmap{Color.RESET}")
    print(f"{Color.RESET}Shows expected positional error between residue pairs.\nLower values = more reliable alignment.{Color.RESET}")
    print(f"{Color.CYAN}{'='*50}{Color.RESET}\n")

    pae = np.load(pae_file)["pae"]
    plt.figure(figsize=(6,5))
    plt.imshow(pae, cmap="viridis", origin="lower")
    plt.colorbar(label="Predicted Aligned Error (Å)")
    plt.title(f"Model {model_id} | PAE Heatmap")
    plt.xlabel("Residue index")
    plt.ylabel("Residue index")
    plt.tight_layout(pad=3.0)
    plt.show()

# Optionally call visualization if prediction succeeded
if os.path.exists(f"/content/boltz_data/{output_dir}/boltz_results_protein/predictions/protein/protein_model_0.cif"):
    visualize_boltz_results(output_dir=output_dir, model_id=0)


In [ ]:
# @title Copy Results to Drive
import shutil, os
from google.colab import drive
from Bio.PDB import MMCIFParser, PDBIO

# ANSI color codes for colored output
class Color:
    CYAN = "\033[96m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    RED = "\033[91m"
    BLUE = "\033[94m"
    MAGENTA = "\033[95m"
    RESET = "\033[0m"
    
# Mount Google Drive
drive.mount('/content/drive')

# Paths
drive_output_dir = f"/content/drive/MyDrive/Boltz2_Results/{output_dir}"
local_output_path = f"/content/boltz_data/{output_dir}"

# Convert CIF to PDB
cif_file = f"{local_output_path}/boltz_results_protein/predictions/protein/protein_model_0.cif"
pdb_file = f"{local_output_path}/{output_dir}.pdb"

parser = MMCIFParser(QUIET=True)
structure = parser.get_structure("prot", cif_file)
io = PDBIO()
io.set_structure(structure)
io.save(pdb_file)

# Remove old folder in Drive if exists
if os.path.exists(drive_output_dir):
    print(f"Removing existing folder {drive_output_dir}")
    shutil.rmtree(drive_output_dir)
    print("Old Drive folder removed.")

# Copy local output folder to Drive
shutil.copytree(local_output_path, drive_output_dir)
print(f"{Color.GREEN}All results copied to Google Drive: {drive_output_dir}{Color.RESET}")
# Copy PDB file separately (optional, just in case)
drive_pdb_file = os.path.join(drive_output_dir, os.path.basename(pdb_file))
shutil.copy(pdb_file, drive_pdb_file)

In [ ]:
# @title Download Results (.zip)
from google.colab import files
from Bio.PDB import MMCIFParser, PDBIO
import shutil
import os

# Local output folder you want to download
local_output_path = f"/content/boltz_data/{output_dir}"

# Convert CIF to PDB
cif_file = f"{local_output_path}/boltz_results_protein/predictions/protein/protein_model_0.cif"
pdb_file = f"{local_output_path}/{output_dir}.pdb"

# Parse CIF and save as PDB
parser = MMCIFParser(QUIET=True)
structure = parser.get_structure("prot", cif_file)
io = PDBIO()
io.set_structure(structure)
io.save(pdb_file)

# Path for the zip file
zip_file = f"/content/{output_dir}.zip"

# Remove previous zip if exists
if os.path.exists(zip_file):
    os.remove(zip_file)

# Create zip of the entire folder
shutil.make_archive(base_name=f"/content/{output_dir}", format='zip', root_dir=local_output_path)

# Download the zip file
files.download(zip_file)

# Success message
print(f"{Color.GREEN}Download successful! All results from '{output_dir}' are saved in '{zip_file}'{Color.RESET}")
